# Data Preprocessing

In [1]:
import pandas as pd

Load Data

In [2]:
df = pd.read_csv("../data/raw/ikman_cars_raw.csv")

df.head()

,url,price,brand,model,year,condition,transmission,body_type,fuel_type,engine_capacity,mileage
0,https://ikman.lk/en/ad/toyota-yaris-2009-for-s...,"Rs 5,975,000",Toyota,Yaris,2009.0,Used,Automatic,Saloon,Petrol,"1,300 cc","135,000 km"
1,https://ikman.lk/en/ad/toyota-land-cruiser-sah...,"Rs 46,800,000",Toyota,Land Cruiser Sahara,2011.0,Used,Automatic,SUV / 4x4,Diesel,"4,600 cc","95,000 km"
2,https://ikman.lk/en/ad/honda-vezel-hybrid-e-he...,"Rs 17,950,000",Honda,Vezel,2026.0,Brand New,Tiptronic,SUV / 4x4,Hybrid,"1,500 cc",0 km
3,https://ikman.lk/en/ad/kia-sonet-2026-for-sale...,"Rs 12,400,000",Kia,Other Model,2026.0,Brand New,Automatic,Hatchback,Petrol,990 cc,0 km
4,https://ikman.lk/en/ad/toyota-land-cruiser-pra...,"Rs 56,000,000",Toyota,Land Cruiser Prado,2024.0,Used,Automatic,SUV / 4x4,Petrol,"2,777 cc","3,200 km"


In [3]:
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10819 entries, 0 to 10818
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              10819 non-null  object 
 1   price            10789 non-null  object 
 2   brand            10789 non-null  object 
 3   model            10789 non-null  object 
 4   year             10789 non-null  float64
 5   condition        10789 non-null  object 
 6   transmission     10789 non-null  object 
 7   body_type        9888 non-null   object 
 8   fuel_type        10789 non-null  object 
 9   engine_capacity  10789 non-null  object 
 10  mileage          10789 non-null  object 
dtypes: float64(1), object(10)
memory usage: 929.9+ KB


,url,price,brand,model,year,condition,transmission,body_type,fuel_type,engine_capacity,mileage
count,10819,10789,10789,10789,10789.000000,10789,10789,9888,10789,10789,10789
unique,9174,1633,61,502,NaN,4,4,7,6,321,1958
top,https://ikman.lk/en/ad/ford-raptor-ranger-ms-r...,"Rs 12,500,000",Toyota,Vezel,NaN,Used,Automatic,SUV / 4x4,Petrol,"1,500 cc",0 km
freq,19,86,3747,560,NaN,6073,8666,3452,7022,2219,2014
mean,NaN,NaN,NaN,NaN,2016.722124,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,10.259259,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1950.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2019.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,NaN,NaN,NaN,NaN,NaN,NaN


Clean Price Column

In [4]:
df["price"] = (
    df["price"]
    .str.replace("Rs", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

df["price"] = pd.to_numeric(df["price"], errors="coerce")

Clean Mileage

In [5]:
df["mileage"] = (
    df["mileage"]
    .str.replace("km", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

df["mileage"].isnull().sum()
df = df.dropna(subset=["mileage"])
df["mileage"] = df["mileage"].astype(int)

Clean Engine Capacity

In [6]:
df["engine_capacity"] = (
    df["engine_capacity"]
    .str.replace("cc", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

df["engine_capacity"].isnull().sum()
df = df.dropna(subset=["engine_capacity"])
df["engine_capacity"] = df["engine_capacity"].astype(int)

Fix Year Column

In [7]:
df["year"].isnull().sum()
df = df.dropna(subset=["year"])
df["year"] = df["year"].astype(int)

Drop url Column

In [8]:
df = df.drop(columns=["url"])

Remove Missing Values

In [9]:
df.isnull().sum()
df = df.dropna(subset=["price", "year", "mileage", "engine_capacity"])

Remove Outliers

In [10]:
df = df[df["year"] >= 1990]
df = df[df["mileage"] < 500000]

Q1 = df['price'].quantile(0.01)
Q3 = df['price'].quantile(0.99)
df = df[(df['price'] >= Q1) & (df['price'] <= Q3)]

df = df[df["engine_capacity"] > 50]

Fill missing body type values

In [11]:
df['body_type'] = df['body_type'].fillna('Unknown')

Feature Engineering - Create car age

In [12]:
current_year = 2026
df["car_age"] = current_year - df["year"]

Save Cleaned Dataset

In [13]:
df.to_csv("../data/processed/cars_cleaned.csv", index=False)